<a href="https://colab.research.google.com/github/Nishok2002/Nishok2002/blob/main/Ingredient_List_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import nltk
import spacy
import plotly as plt
import scipy as sp
import pandas as pd
import en_core_web_sm
import en_core_web_lg
from re import search
from ipywidgets import *
from spacy.matcher import PhraseMatcher
from IPython.display import display, HTML

In [2]:
import spacy

spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
pip install en_core_web_sm

In [4]:
nlp = en_core_web_sm.load()

In [5]:
data = pd.DataFrame(
    ["""RASPBERRY FILLING (INVERT SUGAR, CORN SYRUP, SUGAR, RASPBERRY PUREE, GLYCERIN, 
    MALTODEXTRIN, MODIFIED CORN STARCH, RASPBERRY JUICE CONCENTRATE, SODIUM ALGINATE, 
    METHYLCELLULOSE, VEGETABLE JUICE FOR COLOR (RADISH, CARROT, APPLE, BLACKCURRANT, 
    HIBISCUS CONCENTRATES), MONOCALCIUM PHOSPHATE, XANTHAN GUM, DICALCIUM PHOSPHATE, 
    CITRIC ACID, NATURAL FLAVORS, MALIC ACID), WHOLE GRAIN ROLLED OATS, 
    WHOLE GRAIN WHEAT FLOUR, ENRICHED WHEAT FLOUR (BLEACHED WHEAT FLOUR, 
    MALTED BARLEY FLOUR, NIACIN, REDUCED IRON, THIAMIN MONONITRATE, RIBOFLAVIN, 
    FOLIC ACID), VEGETABLE OIL BLEND (CANOLA, PALM, PALM KERNEL), INVERT SUGAR, 
    SUGAR, GLYCERIN, CONTAINS LESS THAN 2% OF THE FOLLOWING: WHEY, SOLUBLE CORN FIBER, 
    CALCIUM CARBONATE HONEY, WHEAT BRAN, SALT, POTASSIUM BICARBONATE (LEAVENING), 
    SORBITAN MONOSTEARATE, VITAL WHEAT GLUTEN, CORN STARCH, XANTHAN GUM, REDUCED IRON, 
    NIACINAMIDE, PYRIDOXINE HYDROCHLORIDE (VITAMIN B6), DICALCIUM PHOSPHATE, ZINC OXIDE, 
    VITAMIN A PALMITATE, FOLIC ACID, THIAMIN HYDROCHLORIDE (VITAMIN B1), RIBOFLAVIN 
    (VITAMIN B2), CYANOCOBALAMIN (VITAMIN B12), NATURAL FLAVOR, MOLASSES,]""", 
     """MILK, CREAM, SKIM MILK, SUGAR, BLUE FROSTING (CORN SYRUP, 
     HIGH FRUCTOSE CORN SYRUP, WATER, STABILIZERS (FOOD STARCH-MODIFIED, CELLULOSE GUM, 
     DEXTROSE, CARRAGEENAN, GUM ARABIC, CITRIC ACID, TRICALCIUM PHOSPHATE, 
     SILICON DIOXIDE), TITANIUM DIOXIDE FOR COLOR, ARTIFICIAL FLAVOR, CITRIC ACID, 
     BLUE #1), CORN SYRUP, SEQUIN CANDY (SUGAR, CORN STARCH, RICE FLOUR, 
     PARTIALLY HYDROGENATED VEGETABLE OIL (SOYBEAN, COTTONSEED), GUM ARABIC, XANTHAN GUM, 
     CONFECTIONER GLAZE, NATURAL AND ARTIFICIAL FLAVORS, MONO & DIGLYCERIDES, 
     POLYSORBATE 60, TITANIUM DIOXIDE FOR COLOR, YELLOW 5, YELLOW 6, BLUE 1, RED 3, 
     BLUE 1 LAKE), CONTAINS 1% OR LESS OF THE FOLLOWING: NATURAL AND ARTIFICIAL FLAVORS, 
     CAROB BEAN GUM, GUAR GUM, MONO & DIGLYCERIDES, CARRAGEENAN CELLULOSE GEL, 
     CELLULOSE GUM,""", 
     """CREAM, SKIM MILK, LIQUID SUGAR, SEA SALT CARAMEL (CORN SYRUP, WATER, 
     SWEETENED CONDENSED SKIM MILK (SUGAR, WATER, NONFAT MILK SOLIDS), BUTTER (CREAM, 
     SALT), SEA SALT, CARRAGEENAN SODIUM BICARBONATE), MILK, CARAMEL BASE (CORN SYRUP, 
     SUGAR, WATER, NONFAT MILK SOLIDS, BUTTER (CREAM, SALT), SALT), 
     CARAMEL SEA SALT TRUFFLES (SUGAR, COCONUT OIL, CORN SYRUP, SWEETENED CONDENSED MILK 
     (MILK, SUGAR), CREAM, NONFAT MILK, MILK, WATER, COCOA (PROCESSED WITH ALKALI), 
     BUTTER (CREAM, SALT), SEA SALT, SOY LECITHIN, NATURAL FLAVORS), EGG YOLK, 
     CARAMEL COLOR,"""], 
    columns = ["ingredients"])

In [6]:
data["tokenized"] = [nlp(text) for text in data.ingredients]
data

,ingredients,tokenized
0,"RASPBERRY FILLING (INVERT SUGAR, CORN SYRUP, S...","(RASPBERRY, FILLING, (, INVERT, SUGAR, ,, CORN..."
1,"MILK, CREAM, SKIM MILK, SUGAR, BLUE FROSTING (...","(MILK, ,, CREAM, ,, SKIM, MILK, ,, SUGAR, ,, B..."
2,"CREAM, SKIM MILK, LIQUID SUGAR, SEA SALT CARAM...","(CREAM, ,, SKIM, MILK, ,, LIQUID, SUGAR, ,, SE..."


In [7]:
def transform(data, index):
    columns = ["ingredient", "subingredient_1", "subingredient_2", "root_ingredient"]
    new_data = pd.DataFrame(columns=columns)
    ingredient = ''
    subingredient_1 = ''
    subingredient_2 = ''
    paren_count = 0
    to_append = pd.DataFrame(columns=columns)
    for token in data:
        if str(token) ==  '(':
            paren_count = paren_count + 1
        elif str(token) ==  ')':
            paren_count = paren_count - 1

        if str(token) not in ['(', ')', ',', ':']:
            if paren_count == 0:
                ingredient = ingredient + ' ' + str(token)
            elif paren_count == 1:   
                subingredient_1 = subingredient_1 + ' ' + str(token)
            elif paren_count == 2:
                subingredient_2 = subingredient_2 + ' ' + str(token)        

        elif str(token) in [',', ':']:
            if (ingredient != '') | (subingredient_1 != '') | (subingredient_2 != ''):
                new_data=new_data.append(to_append)
                to_append = pd.DataFrame({'ingredient': ingredient.strip(), 
                                          'subingredient_1': subingredient_1.strip(),
                                          'subingredient_2': subingredient_2.strip()},
                                         index=[index])
            if paren_count == 0:
                ingredient = ''
                subingredient_1 = ''
                subingredient_2 = ''
            elif paren_count == 1:
                subingredient_1 = ''
                subingredient_2 = ''
            elif paren_count == 2:
                subingredient_2 = ''
    return new_data

In [8]:
new_data = pd.DataFrame()
for i in [0, 1, 2]:
    new_data = new_data.append(transform(data.tokenized[i], i))
    
new_data

,ingredient,subingredient_1,subingredient_2,root_ingredient
0,RASPBERRY FILLING,INVERT SUGAR,,NaN
0,RASPBERRY FILLING,CORN SYRUP,,NaN
0,RASPBERRY FILLING,SUGAR,,NaN
0,RASPBERRY FILLING,RASPBERRY PUREE,,NaN
0,RASPBERRY FILLING,GLYCERIN,,NaN
...,...,...,...,...
2,CARAMEL SEA SALT TRUFFLES,BUTTER,SALT,NaN
2,CARAMEL SEA SALT TRUFFLES,SEA SALT,,NaN
2,CARAMEL SEA SALT TRUFFLES,SOY LECITHIN,,NaN
2,CARAMEL SEA SALT TRUFFLES,NATURAL FLAVORS,,NaN


In [9]:
def get_root(new_data):
    new_data.root_ingredient = new_data.subingredient_2
    new_data.root_ingredient[new_data.root_ingredient == ''] = new_data[new_data.root_ingredient == ''].subingredient_1
    new_data.root_ingredient[new_data.root_ingredient == ''] = new_data[new_data.root_ingredient == ''].ingredient
    return new_data.root_ingredient

In [10]:
i_data = get_root(new_data)
i_data

0       INVERT SUGAR
0         CORN SYRUP
0              SUGAR
0    RASPBERRY PUREE
0           GLYCERIN
          ...       
2               SALT
2           SEA SALT
2       SOY LECITHIN
2    NATURAL FLAVORS
2           EGG YOLK
Name: root_ingredient, Length: 137, dtype: object

In [11]:
def clean(i_data):
    booleans = [[False if search("CONTAIN", x) != None else True for x in i_data],
               [False if search("WITH ALKALI", x) != None else True for x in i_data]]
    indexes = list(map(all, zip(*booleans)))
    fin_data = i_data[indexes]
    return fin_data 

In [12]:
fin_data = clean(i_data)
fin_data

0       INVERT SUGAR
0         CORN SYRUP
0              SUGAR
0    RASPBERRY PUREE
0           GLYCERIN
          ...       
2               SALT
2           SEA SALT
2       SOY LECITHIN
2    NATURAL FLAVORS
2           EGG YOLK
Name: root_ingredient, Length: 134, dtype: object

In [13]:
print('Here are the items in list 1: \n' + 
      str(list(fin_data[fin_data.index==0].unique())), end = ' ')

Here are the items in list 1: 
['INVERT SUGAR', 'CORN SYRUP', 'SUGAR', 'RASPBERRY PUREE', 'GLYCERIN', 'MALTODEXTRIN', 'MODIFIED CORN STARCH', 'RASPBERRY JUICE CONCENTRATE', 'SODIUM ALGINATE', 'METHYLCELLULOSE', 'RADISH', 'CARROT', 'APPLE', 'BLACKCURRANT', 'HIBISCUS CONCENTRATES', 'MONOCALCIUM PHOSPHATE', 'XANTHAN GUM', 'DICALCIUM PHOSPHATE', 'CITRIC ACID', 'NATURAL FLAVORS', 'MALIC ACID', 'WHOLE GRAIN ROLLED OATS', 'WHOLE GRAIN WHEAT FLOUR', 'BLEACHED WHEAT FLOUR', 'MALTED BARLEY FLOUR', 'NIACIN', 'REDUCED IRON', 'THIAMIN MONONITRATE', 'RIBOFLAVIN', 'FOLIC ACID', 'CANOLA', 'PALM', 'PALM KERNEL', 'WHEY', 'SOLUBLE CORN FIBER', 'CALCIUM CARBONATE HONEY', 'WHEAT BRAN', 'SALT', 'LEAVENING', 'SORBITAN MONOSTEARATE', 'VITAL WHEAT GLUTEN', 'CORN STARCH', 'NIACINAMIDE', 'VITAMIN B6', 'ZINC OXIDE', 'VITAMIN A PALMITATE', 'VITAMIN B1', 'VITAMIN B2', 'VITAMIN B12', 'NATURAL FLAVOR'] 

In [14]:
print('Here are the items in list 2: \n' + 
      str(list(fin_data[fin_data.index==1].unique())), end = ' ')

Here are the items in list 2: 
['MILK', 'CREAM', 'SKIM MILK', 'SUGAR', 'CORN SYRUP', 'HIGH FRUCTOSE CORN SYRUP', 'WATER', 'FOOD STARCH - MODIFIED', 'CELLULOSE GUM', 'DEXTROSE', 'CARRAGEENAN', 'GUM ARABIC', 'CITRIC ACID', 'TRICALCIUM PHOSPHATE', 'SILICON DIOXIDE', 'TITANIUM DIOXIDE FOR COLOR', 'ARTIFICIAL FLAVOR', 'BLUE # 1', 'CORN STARCH', 'RICE FLOUR', 'SOYBEAN', 'COTTONSEED', 'XANTHAN GUM', 'CONFECTIONER GLAZE', 'NATURAL AND ARTIFICIAL FLAVORS', 'MONO & DIGLYCERIDES', 'POLYSORBATE 60', 'YELLOW 5', 'YELLOW 6', 'BLUE 1', 'RED 3', 'BLUE 1 LAKE', 'CAROB BEAN GUM', 'GUAR GUM', 'CARRAGEENAN CELLULOSE GEL'] 

In [15]:
print('Here are the items in list 3: \n' + 
      str(list(fin_data[fin_data.index==2].unique())), end = ' ')

Here are the items in list 3: 
['CREAM', 'SKIM MILK', 'LIQUID SUGAR', 'CORN SYRUP', 'WATER', 'SUGAR', 'NONFAT MILK SOLIDS', 'SALT', 'SEA SALT', 'CARRAGEENAN SODIUM BICARBONATE', 'MILK', 'COCONUT OIL', 'NONFAT MILK', 'SOY LECITHIN', 'NATURAL FLAVORS', 'EGG YOLK'] 

In [16]:
synonyms = pd.DataFrame(columns=["syn1", "syn2"])
ingredient = ''
subingredient = ''
for i in range(1, len(new_data.ingredient)-1):
    if (ingredient != new_data.ingredient.iloc[i]) & (new_data.ingredient.iloc[i+1] != new_data.ingredient.iloc[i]):
        if new_data.subingredient_1.iloc[i] != '':
            synonyms = synonyms.append({"syn1": new_data.ingredient.iloc[i],
                                       "syn2": new_data.subingredient_1.iloc[i]},
                                      ignore_index=[1])
    if (subingredient != new_data.subingredient_1.iloc[i]) & (new_data.subingredient_1.iloc[i+1] != new_data.subingredient_1.iloc[i]):
        if new_data.subingredient_2.iloc[i] != '':
            synonyms = synonyms.append({"syn1": new_data.subingredient_1.iloc[i],
                                        "syn2": new_data.subingredient_2.iloc[i]},
                                       ignore_index=[1])
    ingredient = new_data.ingredient.iloc[i]
    subingredient = new_data.subingredient_1.iloc[i]
    
synonyms

,syn1,syn2
0,POTASSIUM BICARBONATE,LEAVENING
1,PYRIDOXINE HYDROCHLORIDE,VITAMIN B6
2,THIAMIN HYDROCHLORIDE,VITAMIN B1
3,RIBOFLAVIN,VITAMIN B2
4,CYANOCOBALAMIN,VITAMIN B12
5,COCOA,PROCESSED WITH ALKALI


In [17]:
fin_toke = [nlp(text) for text in fin_data]

columns = ["descriptor", "type"]
descriptors = pd.DataFrame(columns=columns)
for i in range(0, len(fin_toke)):
    for token in fin_toke[i]:
        print('%r (%s)' % (str(token), token.dep_))
        descriptors = descriptors.append({"descriptor": token,
                                          "type": token.dep_}, ignore_index=True)

'INVERT' (compound)
'SUGAR' (ROOT)
'CORN' (compound)
'SYRUP' (ROOT)
'SUGAR' (ROOT)
'RASPBERRY' (compound)
'PUREE' (ROOT)
'GLYCERIN' (ROOT)
'MALTODEXTRIN' (ROOT)
'MODIFIED' (compound)
'CORN' (compound)
'STARCH' (ROOT)
'RASPBERRY' (nsubj)
'JUICE' (compound)
'CONCENTRATE' (ROOT)
'SODIUM' (compound)
'ALGINATE' (ROOT)
'METHYLCELLULOSE' (ROOT)
'RADISH' (ROOT)
'CARROT' (ROOT)
'APPLE' (ROOT)
'BLACKCURRANT' (ROOT)
'HIBISCUS' (compound)
'CONCENTRATES' (ROOT)
'MONOCALCIUM' (compound)
'PHOSPHATE' (ROOT)
'XANTHAN' (compound)
'GUM' (ROOT)
'DICALCIUM' (compound)
'PHOSPHATE' (ROOT)
'CITRIC' (compound)
'ACID' (ROOT)
'NATURAL' (compound)
'FLAVORS' (ROOT)
'MALIC' (compound)
'ACID' (ROOT)
'WHOLE' (compound)
'GRAIN' (nsubj)
'ROLLED' (ROOT)
'OATS' (dobj)
'WHOLE' (compound)
'GRAIN' (compound)
'WHEAT' (compound)
'FLOUR' (ROOT)
'BLEACHED' (compound)
'WHEAT' (compound)
'FLOUR' (ROOT)
'MALTED' (compound)
'BARLEY' (compound)
'FLOUR' (ROOT)
'NIACIN' (ROOT)
'REDUCED' (compound)
'IRON' (ROOT)
'THIAMIN' (compound)
'M